In [142]:
G = CartanType('A2')
G.dynkin_diagram()

O---O
1   2   
A2

In [143]:
L = CartanType('A1')
L.dynkin_diagram()

O
1   
A1

In [144]:
crossed_out_nodes = [1]
crossed_out_nodes.sort()
crossed_out_nodes

[1]

In [156]:
G.marked_nodes(crossed_out_nodes).dynkin_diagram()

X---O
1   2   
A2 with node 1 marked

Correction function which converts index of L to that of G.

In [145]:
def correct_index(index: int) -> int:
    for i in range(len(crossed_out_nodes)):
        if index + i < crossed_out_nodes[i]:
            return index + i
    return index + len(crossed_out_nodes)

In [146]:
[ correct_index(k) for k in L.index_set() ]

[2]

In [147]:
R_G = WeylCharacterRing(G)
R_L = WeylCharacterRing(L)

In [148]:
def root_difference_multiplicities(character_ring, highest_weight) -> dict:
    weight_muliplicities = character_ring(highest_weight).weight_multiplicities()
    A = matrix([ vector(sr) for sr in character_ring.simple_roots() ]).transpose()

    result ={}

    for k, v in weight_muliplicities.items():
        Y = vector(k) - vector(highest_weight)
        result[tuple(A.solve_right(Y))] = v
    
    return result

In [149]:
R = PolynomialRing(ZZ, 'x', G.root_system().ambient_space().dimension())
R

Multivariate Polynomial Ring in x0, x1, x2 over Integer Ring

In [159]:
def class_from_weight(weight):
    return sum(R(f'{weight[i]} * x{i}') for i in range(len(weight)))

In [160]:
class_from_weight((1,2,3))

x0 + 2*x1 + 3*x2

The below variable `weight` means a list or tuple of coefficients, so `weight` stands for an actual weight
`sum( weight[i] * fundamental_weight[i] for i)`

In [174]:
weight = (1, 3)
weight_for_L = [ weight[i - 1] for i in set(1..len(weight))-set(crossed_out_nodes)]
weight_for_L

[3]

In [181]:
fws = R_L.fundamental_weights()
fws = [fw for fw in R_L.fundamental_weights()]
weight_for_L = sum( weight_for_L[i] * fws[i] for i in range(L.rank()))
weight_for_L

(3, 0)

In [185]:
R_L(weight_for_L).weight_multiplicities()

{(3, 0): 1, (2, 1): 1, (1, 2): 1, (0, 3): 1}

In [202]:
mul_set = root_difference_multiplicities(R_L, weight_for_L)
mul_set

{(0,): 1, (-1,): 1, (-2,): 1, (-3,): 1}

In [191]:
fws_G = [fw for fw in R_G.fundamental_weights()]; fws_G
weight_for_G = sum( weight[i] * fws_G[i] for i in range(G.rank()))
weight_for_G

(4, 3, 0)

In [203]:

[ weight_for_G + sum(k[i - 1] * R_G.simple_roots()[correct_index(i)] for i in (1..L.rank())) for k,v in mul_set.items() ]



[(4, 3, 0), (4, 2, 1), (4, 1, 2), (4, 0, 3)]

In [204]:
result = {}

for k, v in mul_set.items():
    w = weight_for_G + sum(k[i - 1] * R_G.simple_roots()[correct_index(i)] for i in (1..L.rank()))
    result[w] = v

result

{(4, 3, 0): 1, (4, 2, 1): 1, (4, 1, 2): 1, (4, 0, 3): 1}

In [206]:
def parabolic_weight_muliplicities(weight) -> dict:
    fws_L = [fw for fw in R_L.fundamental_weights()] # conversin from 1-index to 0-index
    weight_for_L = [ weight[i - 1] for i in set(1..len(weight))-set(crossed_out_nodes)] 
    weight_for_L = sum( weight_for_L[i] * fws_L[i] for i in range(L.rank()))

    fws_G = [fw for fw in R_G.fundamental_weights()]; fws_G
    weight_for_G = sum( weight[i] * fws_G[i] for i in range(G.rank()))

    mul_set = root_difference_multiplicities(R_L, weight_for_L)

    result = {}
    for k, v in mul_set.items():
        w = weight_for_G + sum(k[i - 1] * R_G.simple_roots()[correct_index(i)] for i in (1..L.rank()))
        result[w] = v

    return result

In [207]:
parabolic_weight_muliplicities((2,1))

{(3, 1, 0): 1, (3, 0, 1): 1}